In [1]:
import torch
import os
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import torchvision

In [2]:
!nvidia-smi  ### GPU online

Sat Jul 12 01:55:54 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   73C    P8             12W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [3]:
device = torch.device("cuda")

# ResNet50

In [4]:
from torchvision.models import resnet50

In [39]:
model = resnet50(weights="IMAGENET1K_V1") #Instanciamos el modelo
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

## EDA y feature engineering



In [6]:
import kagglehub    ### Importamos para usar nuestro dataset dentro del notebook en Google Colab

# Download latest version
path = kagglehub.dataset_download("paultimothymooney/chest-xray-pneumonia")

print("Ruta de los archivos del Dataset:", path)

Path to dataset files: /kaggle/input/chest-xray-pneumonia


In [45]:
os.listdir("/kaggle/input/chest-xray-pneumonia/chest_xray")

['chest_xray', '__MACOSX', 'val', 'test', 'train']

In [46]:
from pathlib import Path
Ruta_normal_train = Path('/kaggle/input/chest-xray-pneumonia/chest_xray/train/NORMAL')
Ruta_normal_val = Path('/kaggle/input/chest-xray-pneumonia/chest_xray/val/NORMAL')
Ruta_neumonia_train = Path('/kaggle/input/chest-xray-pneumonia/chest_xray/train/PNEUMONIA')
Ruta_neumonia_val = Path('/kaggle/input/chest-xray-pneumonia/chest_xray/val/PNEUMONIA')

Cantidad_train_normal = sum(1 for archivo in Ruta_normal_train.iterdir() if archivo.is_file())
Cantidad_train_neumonia = sum(1 for archivo in Ruta_neumonia_train.iterdir() if archivo.is_file())
Cantidad_val_normal = sum(1 for archivo in Ruta_normal_val.iterdir() if archivo.is_file())
Cantidad_val_neumonia = sum(1 for archivo in Ruta_neumonia_val.iterdir() if archivo.is_file())
print(f"Cantidad =  NORMAL: {Cantidad_train_normal+Cantidad_val_normal}; NEUMONIA: {Cantidad_train_neumonia+Cantidad_val_neumonia}")

Cantidad =  NORMAL: 1349; NEUMONIA: 3883


In [8]:
from torchvision.datasets import ImageFolder #Localizar una carpeta y sus imagenes
from torch.utils.data import DataLoader #Genera un "generador"

In [17]:
from torchvision import transforms #Transformaciones a la imagen
preprocessing = transforms.Compose([
    transforms.Resize(256), # Cambio de tamaño
    transforms.CenterCrop(224), # Centrar imagen
    transforms.ToTensor(), # Transformar en tensor
    transforms.Normalize(  # Normalizar
        mean = [0.485, 0.456, 0.406],
        std = [0.229, 0.224, 0.225])
])

In [18]:
train_folder = ImageFolder("/kaggle/input/chest-xray-pneumonia/chest_xray/train", transform=preprocessing) #Train
val_folder = ImageFolder("/kaggle/input/chest-xray-pneumonia/chest_xray/val", transform=preprocessing) #Val

In [19]:
train_loader = DataLoader(train_folder, batch_size=32)
val_loader = DataLoader(val_folder, batch_size=32)
#Generadores de imagenes con batch size = 32

## Transfer Learning

In [12]:
from torch.nn import CrossEntropyLoss #Loss function
from torch.optim import SGD #Stochastic gradient descent

In [13]:
for params in model.parameters(): # Congelamos parametros
    params.requires_grad = False

In [34]:
model.fc = torch.nn.Linear(model.fc.in_features, 2) # Remplazamos la ultima capa, con una capa de 2 clases
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [31]:
loss_fn = CrossEntropyLoss()
optimizer = SGD(params=model.fc.parameters(), lr=1e-2) #0.01

In [47]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.002)

def evaluate(loader):
    model.eval()
    total, correct, loss_total = 0, 0, 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss_total += loss.item() * images.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    avg_loss = loss_total / total
    accuracy = correct / total
    return avg_loss, accuracy

In [29]:
#Training loop
model.to(device) #Mover el modelo a la GPU
train_losses = []
val_losses = []
Train_Acc = []
Val_Acc = []
epochs = 7 #Seteamos un número de épocas
for epoch in range(epochs):
    model.train() #Modo entrenamiento
    train_loss, total, correct = 0, 0, 0
    for inputs, labels in train_loader:
        #Montar los datos en la GPU
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad() #Limpiar los gradientes
        y_prob = model(inputs) #Forward pass
        loss = loss_fn(y_prob, labels) #Calcular la loss
        loss.backward() #Backward pass
        optimizer.step() #Actualizar los pesos
        train_loss += loss.item() #Sumatoria
        _, predicted = torch.max(y_prob, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    #Promediamos
    train_loss = train_loss / len(train_loader)
    train_Acc = 100*correct/total
    Train_Acc.append(train_Acc)
    train_losses.append(train_loss) #Guardar la loss de training
    model.eval() #Modo evaluación
    val_loss, total, correct = 0, 0, 0
    for inputs, labels in val_loader:
        inputs, labels = inputs.to(device), labels.to(device) #Montar los datos en la GPU
        y_prob = model(inputs) #Forward pass
        loss = loss_fn(y_prob, labels) #Calcular la loss
        val_loss += loss.item()
        _, predicted = torch.max(y_prob, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum()
    #Promediamos
    val_Acc = 100*correct/total
    Val_Acc.append(val_Acc)
    #Promediamos
    val_loss = val_loss / len(val_loader)
    val_losses.append(val_loss) #Guardar la loss de validacion
    print(f"Epoch {epoch}, Train Loss: {train_loss}, Val Loss: {val_loss} /////// accuracy train = {train_Acc}, accuracy val = {val_Acc} "  )

Epoch 0, Train Loss: 0.5172312545355844, Val Loss: 1.3150608539581299 /////// accuracy train = 74.36733128834356, accuracy val = 50.0 
Epoch 1, Train Loss: 0.5308446238743015, Val Loss: 1.3201740980148315 /////// accuracy train = 74.1372699386503, accuracy val = 50.0 
Epoch 2, Train Loss: 0.5318358055561598, Val Loss: 1.320379376411438 /////// accuracy train = 74.1180981595092, accuracy val = 50.0 
Epoch 3, Train Loss: 0.5317972123714313, Val Loss: 1.3201876878738403 /////// accuracy train = 74.1180981595092, accuracy val = 50.0 
Epoch 4, Train Loss: 0.5316769242103846, Val Loss: 1.319964051246643 /////// accuracy train = 74.15644171779141, accuracy val = 50.0 
Epoch 5, Train Loss: 0.5315516445717198, Val Loss: 1.319737434387207 /////// accuracy train = 74.17561349693251, accuracy val = 50.0 
Epoch 6, Train Loss: 0.5314275082086493, Val Loss: 1.3195106983184814 /////// accuracy train = 74.17561349693251, accuracy val = 50.0 
Epoch 7, Train Loss: 0.531304947849066, Val Loss: 1.319283723

KeyboardInterrupt: 

Como se puede observar con transfer learning, los nuevos datos sus loss no van disminuyendo a la 3ra época ya convergió a valores fijos. Esta situación es probable que se dé con la diferencia entre el conjunto entrenado original y las rayos x, por eso usaremos los pesos dados de ImageNet como parámetros iniciales para entrenar los datos nuevos.


## Fine Tuning



In [48]:
for params in model.parameters(): ### Descongelamos los gradiantes
    params.requires_grad = True
model.fc = torch.nn.Linear(model.fc.in_features, 2) # Remplazamos la ultima capa, con una capa de 2 clases
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [ ]:
#Training loop
model.to(device) #Mover el modelo a la GPU
train_losses = []
val_losses = []
Train_Acc = []
Val_Acc = []
epochs = 20 #Seteamos un número de épocas
for epoch in range(epochs):
    model.train() #Modo entrenamiento
    train_loss, total, correct = 0, 0, 0
    for inputs, labels in train_loader:
        #Montar los datos en la GPU
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad() #Limpiar los gradientes
        y_prob = model(inputs) #Forward pass
        loss = loss_fn(y_prob, labels) #Calcular la loss
        loss.backward() #Backward pass
        optimizer.step() #Actualizar los pesos
        train_loss += loss.item() #Sumatoria
        _, predicted = torch.max(y_prob, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    #Promediamos
    train_loss = train_loss / len(train_loader)
    train_Acc = 100*correct/total
    Train_Acc.append(train_Acc)
    train_losses.append(train_loss) #Guardar la loss de training
    model.eval() #Modo evaluación
    val_loss, total, correct = 0, 0, 0
    for inputs, labels in val_loader:
        inputs, labels = inputs.to(device), labels.to(device) #Montar los datos en la GPU
        y_prob = model(inputs) #Forward pass
        loss = loss_fn(y_prob, labels) #Calcular la loss
        val_loss += loss.item()
        _, predicted = torch.max(y_prob, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum()
    #Promediamos
    val_Acc = 100*correct/total
    Val_Acc.append(val_Acc)
    #Promediamos
    val_loss = val_loss / len(val_loader)
    val_losses.append(val_loss) #Guardar la loss de validacion
    print(f"Epoch {epoch}, Train Loss: {train_loss}, Val Loss: {val_loss} /////// accuracy train = {train_Acc}, accuracy val = {val_Acc} "  )

Epoch 0, Train Loss: 0.12322902962411154, Val Loss: 3.540546417236328 /////// accuracy train = 95.41794478527608, accuracy val = 50.0 
Epoch 1, Train Loss: 0.39122360452803917, Val Loss: 2.8784782886505127 /////// accuracy train = 85.56365030674847, accuracy val = 50.0 
Epoch 2, Train Loss: 0.4195743319949855, Val Loss: 2.6483213901519775 /////// accuracy train = 83.70398773006134, accuracy val = 50.0 
Epoch 3, Train Loss: 0.4175692005597192, Val Loss: 2.3676698207855225 /////// accuracy train = 85.35276073619632, accuracy val = 50.0 
